In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import statsmodels.api as sm
import seaborn as sns
from datetime import datetime

In [116]:
#df = pd.read_excel(r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\displacements\unhcr-prmn-displacement-dataset-3.xlsx")
df = pd.read_excel(r"/home/sara/Documenti/GitHub/Climate-and-conflict/displacements/UNHCR-PRMN-Displacement-Dataset - Somalia.xlsx")
#df_d = df[df['Reason']=='Drought related']

In [117]:
path = "./som_adm_ocha_itos_20230308_shp/" 
#states_gdf = gpd.read_file(path+"som_admbnda_adm1_ocha_20230308.shp") #sdn_admbnda_adm1_cbs_nic_ssa_20200831.shp")
states_gdf = gpd.read_file(r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/som_adm_ocha_itos_20230308_shp/som_admbnda_adm1_ocha_20230308.shp")

In [118]:
df.head()

,Month End,Year Week,Current (Arrival) Region,Current (Arrival) District,Previous (Departure) Region,Previous (Departure) District,Reason,Current (Arrival) Priority Need,Number of Individuals
0,30/04/2016,201614,Awdal,Borama,Awdal,Borama,Drought related,Food,6
1,30/04/2016,201617,Awdal,Borama,Awdal,Borama,Drought related,Livelihood Support,7
2,30/04/2016,201616,Awdal,Borama,Awdal,Borama,Flood,Food,5
3,30/04/2016,201614,Awdal,Borama,Awdal,Lughaye,Drought related,Food,8
4,31/03/2016,201611,Awdal,Borama,Awdal,Lughaye,Drought related,Food,9


In [119]:
v= df["Month End"]
v=v.values
dt = [datetime.strptime(v[i], "%d/%m/%Y") for i in range(len(v))]
q=[]

for i in range(len(dt)):
    q.append(datetime.timestamp(dt[i]))
    
df.insert(loc=3, column='date_timestamp', value=q)
df = df.sort_values("date_timestamp")

df['Month End'] = pd.to_datetime(df['Month End'], dayfirst=True)
#df=df.set_index('Month End')

In [120]:
df.head()

,Month End,Year Week,Current (Arrival) Region,date_timestamp,Current (Arrival) District,Previous (Departure) Region,Previous (Departure) District,Reason,Current (Arrival) Priority Need,Number of Individuals
2602,2016-01-31,201602,Woqooyi Galbeed,1.454195e+09,Hargeysa,Hiraan,Belet Weyne,Drought related,Food,6
1316,2016-01-31,201603,Gedo,1.454195e+09,Baardheere,Gedo,Garbahaarey,Conflict/Insecurity,Food,795
1317,2016-01-31,201604,Gedo,1.454195e+09,Baardheere,Gedo,Garbahaarey,Conflict/Insecurity,Food,168
1319,2016-01-31,201603,Gedo,1.454195e+09,Belet Xaawo,Gedo,Belet Xaawo,Conflict/Insecurity,Food,1440
1322,2016-01-31,201603,Gedo,1.454195e+09,Belet Xaawo,Gedo,Belet Xaawo,Conflict/Insecurity,Other,660


In [121]:
# Aggregate the datetime objects by month
aggregated_data = df.groupby([pd.Grouper(key='Month End', freq='M'),'Previous (Departure) Region', 'Current (Arrival) Region'])['Number of Individuals'].sum().to_frame()

In [124]:
dates = aggregated_data.index.get_level_values('Month End').unique()
districts = aggregated_data.index.get_level_values('Previous (Departure) Region').unique()
all_combinations = pd.MultiIndex.from_product([dates, districts,districts], names=['time', 'Previous (Departure) Region','Current (Arrival) Region'])

disp_data = aggregated_data.reindex(all_combinations, fill_value=0).reset_index()   

In [129]:
#pivot table
disp_matxs = disp_data.pivot_table(index=['time','Previous (Departure) Region'], columns='Current (Arrival) Region', values='Number of Individuals', aggfunc='sum')

In [130]:
disp_matxs

Current (Arrival) Region                Awdal  Bakool  Banadir  Bari  Bay  \
time       Previous (Departure) Region                                      
2016-01-31 Awdal                            0       0        0     0    0   
           Bakool                           7       0        0     0   14   
           Banadir                         26       0     6441    23    0   
           Bari                             0       0        0     8    6   
           Bay                             13       0        0     0  164   
...                                       ...     ...      ...   ...  ...   
2023-08-31 Nugaal                           0       0        0     8    0   
           Sanaag                           0       0        0   385    0   
           Sool                             0       0        0   173    0   
           Togdheer                         0       0        0     0    0   
           Woqooyi Galbeed                  0       0        0     0    0   

Current (Arrival) Region                Galgaduud  Gedo  Hiraan  Lower Juba  \
time       Previous (Departure) Region                                        
2016-01-31 Awdal                                0     0       0           0   
           Bakool                               0     3       0           0   
           Banadir                              0     0       0           0   
           Bari                                 0     0       0           0   
           Bay                                  0     0       0           0   
...                                           ...   ...     ...         ...   
2023-08-31 Nugaal                               0     0       0           0   
           Sanaag                               0     0       0           0   
           Sool                                 0     0       0           0   
           Togdheer                             0     0       0           0   
           Woqooyi Galbeed                      0     0       0           0   

Current (Arrival) Region                Lower Shabelle  Middle Juba  \
time       Previous (Departure) Region                                
2016-01-31 Awdal                                     0            0   
           Bakool                                    0            0   
           Banadir                                   0            0   
           Bari                                      0            0   
           Bay                                       0          138   
...                                                ...          ...   
2023-08-31 Nugaal                                    0            0   
           Sanaag                                    0            0   
           Sool                                      0            0   
           Togdheer                                  0            0   
           Woqooyi Galbeed                           0            0   

Current (Arrival) Region                Middle Shabelle  Mudug  Nugaal  \
time       Previous (Departure) Region                                   
2016-01-31 Awdal                                      0      0       0   
           Bakool                                     0      0       0   
           Banadir                                    0     57       0   
           Bari                                       0     13       0   
           Bay                                        0     19       0   
...                                                 ...    ...     ...   
2023-08-31 Nugaal                                     0      0    4732   
           Sanaag                                     0      0       0   
           Sool                                       0      0     647   
           Togdheer                                   0      0       0   
           Woqooyi Galbeed                            0      0       0   

Current (Arrival) Region                Sanaag  Sool  Togdheer  \
time       Previo

In [127]:
arrival = disp_data.groupby(['time','Current (Arrival) Region'])['Number of Individuals'].sum()

In [128]:
arrival

time        Current (Arrival) Region
2016-01-31  Awdal                          89
            Bakool                          0
            Banadir                      7018
            Bari                           75
            Bay                           184
                                        ...  
2023-08-31  Nugaal                       6236
            Sanaag                      12176
            Sool                           54
            Togdheer                      393
            Woqooyi Galbeed               169
Name: Number of Individuals, Length: 1656, dtype: int64

In [ ]:
#add rows with missing months and fill with 0
idx = pd.date_range('2016-01-31', '2022-12-31')
aggregated_data = aggregated_data.reindex(idx, fill_value=0)


ValueError: cannot include dtype 'M' in a buffer